In [21]:
import qrcode
import random
from datetime import datetime, timedelta
import heapq
import gradio as gr
import io
from PIL import Image
from openai import OpenAI
import os

class AirportApp:
    def __init__(self):
        self.users = {}
        self.tickets = {}
        self.flight_schedules = {}
        self.check_in_queue = []
        
        # Set up OpenAI client
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        if not self.client.api_key:
            raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

    def create_user(self, user_id, name):
        qr = qrcode.QRCode(version=1, box_size=10, border=5)
        qr.add_data(user_id)
        qr.make(fit=True)
        qr_img = qr.make_image(fill_color="black", back_color="white")
        
        img_byte_arr = io.BytesIO()
        qr_img.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()
        
        self.users[user_id] = {
            "name": name,
            "qr_code": img_byte_arr
        }
        return f"User {name} created with ID {user_id}"

    def create_ticket(self, user_id, flight_number):
        if user_id not in self.users:
            return f"User {user_id} not found"
        if flight_number not in self.flight_schedules:
            return f"Flight {flight_number} not found"
        
        ticket_number = f"T{random.randint(1000, 9999)}"
        self.tickets[ticket_number] = {
            "user_id": user_id,
            "flight_number": flight_number,
            "status": "Check-in"
        }
        return f"Ticket {ticket_number} created for user {user_id} on flight {flight_number}"

    def update_flight_schedule(self, flight_number, status, departure_time):
        self.flight_schedules[flight_number] = {
            "status": status,
            "departure_time": departure_time
        }
        return f"Flight {flight_number} updated: {status}, departure at {departure_time}"

    def get_flight_info(self, flight_number):
        if flight_number in self.flight_schedules:
            info = self.flight_schedules[flight_number]
            return f"Flight {flight_number}: {info['status']}, departure at {info['departure_time']}"
        return f"Flight {flight_number} not found"

    def add_to_check_in_queue(self, user_id, flight_number):
        if flight_number not in self.flight_schedules:
            return f"Flight {flight_number} not found"
        
        departure_time = datetime.strptime(self.flight_schedules[flight_number]['departure_time'], "%Y-%m-%d %H:%M:%S")
        current_time = datetime.now()
        # departure_time = datetime.strptime(departure_time, "%Y-%m-%d %H:%M:%S")

        time_until_departure = (departure_time - current_time).total_seconds() / 60  # in minutes
        
        heapq.heappush(self.check_in_queue, (time_until_departure, user_id, flight_number))
        return f"User {user_id} added to check-in queue for flight {flight_number}"

    def process_check_in_queue(self, num_to_process=5):
        processed = []
        for _ in range(min(num_to_process, len(self.check_in_queue))):
            if self.check_in_queue:
                time_until_departure, user_id, flight_number = heapq.heappop(self.check_in_queue)
                processed.append(f"User {user_id} for flight {flight_number} (Departure in {time_until_departure:.2f} minutes)")
        return "\n".join(processed) if processed else "No users in the queue"

    def genai_bot(self, query):
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an AI assistant for an airport. Provide helpful information about the airport and its services."},
                    {"role": "user", "content": query}
                ]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"An error occurred while calling the OpenAI API: {str(e)}")
            return "I'm sorry, but I'm having trouble connecting to my knowledge base right now. Please try again later."

app = AirportApp()

def create_user_interface(user_id, name):
    result = app.create_user(user_id, name)
    qr_code = Image.open(io.BytesIO(app.users[user_id]["qr_code"])) if user_id in app.users else None
    return result, qr_code

def create_ticket_interface(user_id, flight_number):
    return app.create_ticket(user_id, flight_number)

def update_flight_interface(flight_number, status, departure_time):
    return app.update_flight_schedule(flight_number, status, departure_time)

def get_flight_info_interface(flight_number):
    return app.get_flight_info(flight_number)

def add_to_queue_interface(user_id, flight_number):
    return app.add_to_check_in_queue(user_id, flight_number)

def process_queue_interface(num_to_process):
    return app.process_check_in_queue(int(num_to_process))

def chatbot_interface(message, history):
    history = history or []
    bot_response = app.genai_bot(message)
    history.append((message, bot_response))
    return history

with gr.Blocks() as interface:
    gr.Markdown("# Airport Productivity App")
    
    with gr.Tab("User Management"):
        gr.Markdown("## Create User")
        user_id_input = gr.Textbox(label="User ID")
        name_input = gr.Textbox(label="Name")
        create_user_button = gr.Button("Create User")
        user_output = gr.Textbox(label="Result")
        qr_output = gr.Image(label="QR Code")
        create_user_button.click(create_user_interface, inputs=[user_id_input, name_input], outputs=[user_output, qr_output])

    with gr.Tab("Ticket Management"):
        gr.Markdown("## Create Ticket")
        ticket_user_id = gr.Textbox(label="User ID")
        ticket_flight_number = gr.Textbox(label="Flight Number")
        create_ticket_button = gr.Button("Create Ticket")
        ticket_output = gr.Textbox(label="Result")
        create_ticket_button.click(create_ticket_interface, inputs=[ticket_user_id, ticket_flight_number], outputs=ticket_output)

    with gr.Tab("Flight Management"):
        gr.Markdown("## Update Flight Schedule")
        flight_number = gr.Textbox(label="Flight Number")
        flight_status = gr.Textbox(label="Status")
        departure_time = gr.Textbox(label="Departure Time (YYYY-MM-DD HH:MM:SS)")
        update_flight_button = gr.Button("Update Flight")
        flight_output = gr.Textbox(label="Result")
        update_flight_button.click(update_flight_interface, inputs=[flight_number, flight_status, departure_time], outputs=flight_output)

        gr.Markdown("## Get Flight Info")
        info_flight_number = gr.Textbox(label="Flight Number")
        get_info_button = gr.Button("Get Info")
        info_output = gr.Textbox(label="Flight Information")
        get_info_button.click(get_flight_info_interface, inputs=info_flight_number, outputs=info_output)

    with gr.Tab("Check-in Queue"):
        gr.Markdown("## Add to Check-in Queue")
        queue_user_id = gr.Textbox(label="User ID")
        queue_flight_number = gr.Textbox(label="Flight Number")
        add_to_queue_button = gr.Button("Add to Queue")
        queue_add_output = gr.Textbox(label="Result")
        add_to_queue_button.click(add_to_queue_interface, inputs=[queue_user_id, queue_flight_number], outputs=queue_add_output)

        gr.Markdown("## Process Check-in Queue")
        num_to_process = gr.Slider(minimum=1, maximum=10, step=1, label="Number of Users to Process")
        process_queue_button = gr.Button("Process Queue")
        queue_process_output = gr.Textbox(label="Processed Users")
        process_queue_button.click(process_queue_interface, inputs=num_to_process, outputs=queue_process_output)

    with gr.Tab("Airport Assistant"):
        gr.Markdown("## Chat with Airport Assistant")
        chatbot = gr.Chatbot()
        msg = gr.Textbox()
        clear = gr.Button("Clear")

        msg.submit(chatbot_interface, inputs=[msg, chatbot], outputs=chatbot)
        clear.click(lambda: [], None, chatbot, queue=False)

# interface.launch()

IMPORTANT: You are using gradio version 4.12.0, however version 4.29.0 is available, please upgrade.
--------


In [22]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://1e2fc485f74b4aa5e2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## qr code

In [2]:
import qrcode
import random
from datetime import datetime, timedelta
import heapq
import gradio as gr
import io
from PIL import Image
from openai import OpenAI
import os
import torch
from diffusers import (
    StableDiffusionControlNetImg2ImgPipeline,
    ControlNetModel,
    DPMSolverMultistepScheduler,
)

class AirportApp:
    def __init__(self):
        print("Initializing AirportApp...")
        self.users = {}
        self.tickets = {}
        self.flight_schedules = {}
        self.check_in_queue = []
        
        # Set up OpenAI client
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        if not self.client.api_key:
            raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

        # Set up QR code generator
        self.qrcode_generator = qrcode.QRCode(
            version=1,
            error_correction=qrcode.ERROR_CORRECT_H,
            box_size=10,
            border=4,
        )
        
        print("Loading AI models...")
        self.load_ai_models()
        print("AI models loaded successfully.")

    def load_ai_models(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")

        self.controlnet = ControlNetModel.from_pretrained(
            "DionTimmer/controlnet_qrcode-control_v1p_sd15", torch_dtype=torch.float16
        ).to(device)
        
        self.pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
            "runwayml/stable-diffusion-v1-5",
            controlnet=self.controlnet,
            safety_checker=None,
            torch_dtype=torch.float16,
        ).to(device)
        
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(self.pipe.scheduler.config, use_karras=True, algorithm_type="sde-dpmsolver++")

    def create_user(self, user_id, name, qr_prompt):
        # Generate basic QR code
        self.qrcode_generator.clear()
        self.qrcode_generator.add_data(user_id)
        self.qrcode_generator.make(fit=True)
        qrcode_image = self.qrcode_generator.make_image(fill_color="black", back_color="white")
        
        # Generate AI QR code
        ai_qrcode = self.generate_ai_qrcode(qrcode_image, qr_prompt)
        
        # Save the AI-generated QR code
        img_byte_arr = io.BytesIO()
        ai_qrcode.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()
        
        self.users[user_id] = {
            "name": name,
            "qr_code": img_byte_arr
        }
        return f"User {name} created with ID {user_id}. AI QR code generated successfully."

    def generate_ai_qrcode(self, qrcode_image, qr_prompt):
        qrcode_image = self.resize_for_condition_image(qrcode_image, 768)
        
        generator = torch.manual_seed(random.randint(0, 2**32 - 1))
        out = self.pipe(
            prompt=qr_prompt,
            negative_prompt="ugly, disfigured, low quality, blurry, nsfw",
            image=qrcode_image,
            control_image=qrcode_image,
            width=768,
            height=768,
            guidance_scale=7.5,
            controlnet_conditioning_scale=1.1,
            generator=generator,
            strength=0.9,
            num_inference_steps=40,
        )
        return out.images[0]

    def resize_for_condition_image(self, input_image: Image.Image, resolution: int):
        input_image = input_image.convert("RGB")
        W, H = input_image.size
        k = float(resolution) / min(H, W)
        H *= k
        W *= k
        H = int(round(H / 64.0)) * 64
        W = int(round(W / 64.0)) * 64
        img = input_image.resize((W, H), resample=Image.LANCZOS)
        return img

    def create_ticket(self, user_id, flight_number):
        if user_id not in self.users:
            return f"User {user_id} not found"
        if flight_number not in self.flight_schedules:
            return f"Flight {flight_number} not found"
        
        ticket_number = f"T{random.randint(1000, 9999)}"
        self.tickets[ticket_number] = {
            "user_id": user_id,
            "flight_number": flight_number,
            "status": "Check-in"
        }
        return f"Ticket {ticket_number} created for user {user_id} on flight {flight_number}"

    def update_flight_schedule(self, flight_number, status, departure_time):
        self.flight_schedules[flight_number] = {
            "status": status,
            "departure_time": departure_time
        }
        return f"Flight {flight_number} updated: {status}, departure at {departure_time}"

    def get_flight_info(self, flight_number):
        if flight_number in self.flight_schedules:
            info = self.flight_schedules[flight_number]
            return f"Flight {flight_number}: {info['status']}, departure at {info['departure_time']}"
        return f"Flight {flight_number} not found"

    def add_to_check_in_queue(self, user_id, flight_number):
        if flight_number not in self.flight_schedules:
            return f"Flight {flight_number} not found"
        
        departure_time = datetime.strptime(self.flight_schedules[flight_number]['departure_time'], "%Y-%m-%d %H:%M:%S")
        current_time = datetime.now()
        time_until_departure = (departure_time - current_time).total_seconds() / 60  # in minutes
        
        heapq.heappush(self.check_in_queue, (time_until_departure, user_id, flight_number))
        return f"User {user_id} added to check-in queue for flight {flight_number}"

    def process_check_in_queue(self, num_to_process=5):
        processed = []
        for _ in range(min(num_to_process, len(self.check_in_queue))):
            if self.check_in_queue:
                time_until_departure, user_id, flight_number = heapq.heappop(self.check_in_queue)
                processed.append(f"User {user_id} for flight {flight_number} (Departure in {time_until_departure:.2f} minutes)")
        return "\n".join(processed) if processed else "No users in the queue"

    def genai_bot(self, query):
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are an AI assistant for an airport. Provide helpful information about the airport and its services."},
                    {"role": "user", "content": query}
                ]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"An error occurred while calling the OpenAI API: {str(e)}")
            return "I'm sorry, but I'm having trouble connecting to my knowledge base right now. Please try again later."

print("Creating AirportApp instance...")
app = AirportApp()
print("AirportApp instance created successfully.")

def create_user_interface(user_id, name, qr_prompt):
    result = app.create_user(user_id, name, qr_prompt)
    qr_code = Image.open(io.BytesIO(app.users[user_id]["qr_code"])) if user_id in app.users else None
    return result, qr_code

def create_ticket_interface(user_id, flight_number):
    return app.create_ticket(user_id, flight_number)

def update_flight_interface(flight_number, status, departure_time):
    return app.update_flight_schedule(flight_number, status, departure_time)

def get_flight_info_interface(flight_number):
    return app.get_flight_info(flight_number)

def add_to_queue_interface(user_id, flight_number):
    return app.add_to_check_in_queue(user_id, flight_number)

def process_queue_interface(num_to_process):
    return app.process_check_in_queue(int(num_to_process))

def chatbot_interface(message, history):
    history = history or []
    bot_response = app.genai_bot(message)
    history.append((message, bot_response))
    return history



Creating AirportApp instance...
Initializing AirportApp...
Loading AI models...
Using device: cuda


Loading pipeline components...: 100%|██████████| 6/6 [00:01<00:00,  5.73it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


AI models loaded successfully.
AirportApp instance created successfully.


In [4]:
print("Setting up Gradio interface...")
with gr.Blocks() as interface:
    gr.Markdown("# Airport Productivity App")
    
    with gr.Tab("User Management"):
        gr.Markdown("## Create User")
        user_id_input = gr.Textbox(label="User ID")
        name_input = gr.Textbox(label="Name")
        qr_prompt_input = gr.Textbox(label="QR Code Art Prompt", 
                                     placeholder="Describe the art style for the QR code")
        create_user_button = gr.Button("Create User")
        user_output = gr.Textbox(label="Result")
        qr_output = gr.Image(label="AI-Generated QR Code")
        create_user_button.click(create_user_interface, 
                                 inputs=[user_id_input, name_input, qr_prompt_input], 
                                 outputs=[user_output, qr_output])

    with gr.Tab("Ticket Management"):
        gr.Markdown("## Create Ticket")
        ticket_user_id = gr.Textbox(label="User ID")
        ticket_flight_number = gr.Textbox(label="Flight Number")
        create_ticket_button = gr.Button("Create Ticket")
        ticket_output = gr.Textbox(label="Result")
        create_ticket_button.click(create_ticket_interface, inputs=[ticket_user_id, ticket_flight_number], outputs=ticket_output)

    with gr.Tab("Flight Management"):
        gr.Markdown("## Update Flight Schedule")
        flight_number = gr.Textbox(label="Flight Number")
        flight_status = gr.Textbox(label="Status")
        departure_time = gr.Textbox(label="Departure Time (YYYY-MM-DD HH:MM:SS)")
        update_flight_button = gr.Button("Update Flight")
        flight_output = gr.Textbox(label="Result")
        update_flight_button.click(update_flight_interface, inputs=[flight_number, flight_status, departure_time], outputs=flight_output)

        gr.Markdown("## Get Flight Info")
        info_flight_number = gr.Textbox(label="Flight Number")
        get_info_button = gr.Button("Get Info")
        info_output = gr.Textbox(label="Flight Information")
        get_info_button.click(get_flight_info_interface, inputs=info_flight_number, outputs=info_output)

    with gr.Tab("Check-in Queue"):
        gr.Markdown("## Add to Check-in Queue")
        queue_user_id = gr.Textbox(label="User ID")
        queue_flight_number = gr.Textbox(label="Flight Number")
        add_to_queue_button = gr.Button("Add to Queue")
        queue_add_output = gr.Textbox(label="Result")
        add_to_queue_button.click(add_to_queue_interface, inputs=[queue_user_id, queue_flight_number], outputs=queue_add_output)

        gr.Markdown("## Process Check-in Queue")
        num_to_process = gr.Slider(minimum=1, maximum=10, step=1, label="Number of Users to Process")
        process_queue_button = gr.Button("Process Queue")
        queue_process_output = gr.Textbox(label="Processed Users")
        process_queue_button.click(process_queue_interface, inputs=num_to_process, outputs=queue_process_output)

    with gr.Tab("Airport Assistant"):
        gr.Markdown("## Chat with Airport Assistant")
        chatbot = gr.Chatbot()
        msg = gr.Textbox()
        clear = gr.Button("Clear")

        msg.submit(chatbot_interface, inputs=[msg, chatbot], outputs=chatbot)
        clear.click(lambda: [], None, chatbot, queue=False)

print("Launching Gradio interface...")
interface.launch(share=True)
print("Gradio interface launched.")

Setting up Gradio interface...
Launching Gradio interface...
Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.12.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://f1eae4bfd690a6c85f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Gradio interface launched.


100%|██████████| 36/36 [00:08<00:00,  4.26it/s]
